In [103]:
import pandas as pd


df=pd.read_csv("../results/grid_search_results.csv")

df

,pop_size,generations,selection,crossover,mutation,cx_rate,mut_rate,elitism_percent,runs,early_stop_gens,delta,avg_fitness,max_fitness,avg_generations,max_generations,std_fitness,std_generations,best_solution,fitness_progress
0,10,50,tournament,table_preserve,inversion,0.7,0.9,0.00,3,3,250,73400.000000,78400.0,24.000000,34,4255.192906,7.257180,"[[19, 0, 35, 37, 60, 34, 36, 1], [13, 3, 2, 10...","[38000.0, 40533.333333333336, 42066.6666666666..."
1,10,50,tournament,greedy_merge,inversion,0.5,0.9,0.05,3,3,250,73333.333333,82200.0,34.333333,43,10489.783389,7.363574,"[[33, 34, 42, 18, 57, 55, 54, 35], [10, 40, 29...","[36866.666666666664, 38800.0, 38200.0, 42600.0..."
2,10,50,ranking,table_preserve,inversion,0.9,0.9,0.00,3,3,250,71800.000000,81200.0,31.000000,50,8021.637406,13.441230,"[[1, 25, 26, 39, 0, 14, 16, 40], [5, 9, 23, 31...","[39000.0, 44300.0, 45500.0, 45000.0, 49233.333..."
3,10,50,tournament,table_preserve,inversion,0.9,0.8,0.05,3,3,250,71433.333333,83700.0,13.666667,17,8949.612034,2.867442,"[[47, 5, 18, 17, 46, 15, 16, 51], [53, 2, 3, 7...","[41500.0, 42200.0, 48600.0, 55000.0, 58600.0, ..."
4,10,50,ranking,table_preserve,inversion,0.7,0.8,0.00,3,3,250,70866.666667,79600.0,19.666667,29,9168.545262,7.363574,"[[17, 23, 5, 0, 15, 24, 1, 28], [60, 61, 7, 63...","[43466.666666666664, 45066.666666666664, 47533..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,10,50,ranking,greedy_merge,inversion,0.9,0.5,0.05,3,3,250,39133.333333,41900.0,21.333333,32,2471.616655,8.993825,"[[18, 33, 35, 39, 19, 8, 63, 57], [16, 62, 17,...","[34800.0, 38933.333333333336, 35266.6666666666..."
452,10,50,boltzmann,table_preserve,inversion,0.8,0.6,0.00,3,3,250,38933.333333,43000.0,3.666667,4,2876.726535,0.471405,"[[45, 48, 1, 2, 15, 12, 8, 38], [40, 39, 18, 1...","[37866.666666666664, 38933.333333333336, 38933..."
453,10,50,ranking,greedy_merge,inversion,0.7,0.9,0.05,3,3,250,38466.666667,40800.0,36.000000,50,1651.934892,19.798990,"[[53, 2, 37, 10, 36, 31, 32, 62], [60, 43, 57,...","[36966.666666666664, 35933.333333333336, 41266..."
454,10,50,ranking,greedy_merge,inversion,0.6,0.7,0.05,3,3,250,38266.666667,39900.0,9.000000,17,1155.662388,5.887841,"[[5, 22, 25, 37, 7, 49, 28, 42], [59, 19, 50, ...","[39400.0, 37333.333333333336, 33966.6666666666..."


In [102]:
max(df["avg_generations"])

27.4

In [ ]:
for i in ["selection","crossover","mutation","cx_rate",	"mut_rate",	"elitism_percent","avg_generations"]:
    print(df.head(100)[i].value_counts())

avg_generations
12.600000    3
13.733333    3
13.033333    3
12.533333    3
12.566667    3
            ..
11.933333    1
21.266667    1
11.633333    1
12.966667    1
11.000000    1
Name: count, Length: 75, dtype: int64


**Conclusions:**
* The best overall performance was achieved with:
    * Selection: tournament
    * Crossover: merge
    * Mutation: multiple_point
     * Elitism: enabled (True)
* These configurations showed:
    * A steady and significant increase in fitness over generations.
    * The highest final fitness values among all tested combinations.

**Note:** Elitism and more complex mutation strategies (like multiple_point) play a critical role in ensuring consistent improvement and better final results.

In [ ]:
summary = pd.DataFrame([{
    "Selection": r["selection"],
    "Crossover": r["crossover"],
    "Mutation": r["mutation"],
    "Elitism": r["elitism"],
    "Final Score": r["final_score"]
} for r in results])

summary = summary.sort_values(by="Final Score", ascending=False)
summary.reset_index(drop=True, inplace=True)
summary


,Selection,Crossover,Mutation,Elitism,Final Score
0,tournament,merge,one_point,True,37800
1,roulette,merge,one_point,True,35482
2,tournament,merge,multiple_point,True,30025
3,tournament,merge,swap,True,28590
4,roulette,group,one_point,True,27800
5,roulette,merge,multiple_point,True,27717
6,roulette,merge,swap,True,27265
7,roulette,group,multiple_point,True,25300
8,tournament,group,swap,True,25000
9,roulette,group,swap,True,24940


**Conclusions:**
* Best score: tournament + merge + one_point + elitism=True -> 37800
* worse score: ranking + merge + one_point + elitism=False -> 12600
* Generally, ranking selection without elitism performed poorly